<a href="https://colab.research.google.com/github/WildAlex37/g_search_lite/blob/main/G_search_lite_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Récupération de la Data et cleaning


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import os
pd.options.display.max_columns = None

In [17]:
import os
import json
import time
import requests
import pandas as pd
from datetime import datetime, timedelta

# IGDB API credentials
CLIENT_ID = "v4dlufpsiskkalr4ht57stnpxeh3lo"
CLIENT_SECRET = "8a1okhz512zn4lenv42jzt30c24gd9"

# IGDB API endpoints
AUTH_URL = 'https://id.twitch.tv/oauth2/token'
API_ENDPOINT = 'https://api.igdb.com/v4/games'

# Define the desired platforms
DESIRED_PLATFORMS = ['PlayStation 5', 'Xbox Series', 'Nintendo Switch']

def get_access_token():
    auth_params = {
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': 'client_credentials'
    }
    auth_response = requests.post(AUTH_URL, params=auth_params)
    return auth_response.json()['access_token']

def fetch_games(access_token, time_back: int=365, offset=0):
    headers = {
        'Client-ID': CLIENT_ID,
        'Authorization': f'Bearer {access_token}',
    }

    # Calculate the timestamp for 10 years ago
    limit_time = int((datetime.now() - timedelta(days=time_back)).timestamp())
    current_time = int(datetime.now().timestamp())

    body = f"""
    fields name, first_release_date, release_dates.date, genres.name, platforms.name, summary, cover.url, rating, rating_count;
    where first_release_date >= {limit_time} & release_dates.date <= {current_time};
    sort first_release_dates.date desc;
    limit 500;
    """

    response = requests.post(API_ENDPOINT, headers=headers, data=body)
    return response.json()

def filter_games_by_platforms(games):
    filtered_games = []
    for game in games:
        if 'platforms' in game:
            platforms = [platform['name'] for platform in game['platforms']]
            if any(platform in platforms for platform in DESIRED_PLATFORMS):
                filtered_games.append(game)
    return filtered_games

def get_games(time_back: int):
    access_token = get_access_token()
    all_games = []
    offset = 0

    while True:
        games = fetch_games(access_token, time_back, offset)
        if not games:
            break

        filtered_games = filter_games_by_platforms(games)
        all_games.extend(filtered_games)
        print(f"Fetched {len(games)} games. Filtered to {len(filtered_games)} games. Total: {len(all_games)}")

        if len(games) < 500:
            break

        offset += 500
        time.sleep(0.3)  # Respect rate limits

    # Save all games to a JSON file
    print(f"Scraped a total of {len(all_games)} games from the last {time_back} days for desired platforms.")
    return all_games

if __name__ == "__main__":
    games = get_games(365)
    # Convert the games to a DataFrame
    df = pd.DataFrame(games)
    df.to_json('games_entry.json', orient='records', lines=True)  # Save to JSON
    genres_dummies = pd.get_dummies(df['genres'].apply(lambda x: '||'.join(x) if isinstance(x, list) else x), sep='||')
    df = pd.concat([df, genres_dummies], axis=1)
    df.to_parquet('df.parquet')


Fetched 500 games. Filtered to 166 games. Total: 166
Fetched 500 games. Filtered to 166 games. Total: 332
Fetched 500 games. Filtered to 166 games. Total: 498
Fetched 500 games. Filtered to 166 games. Total: 664
Fetched 500 games. Filtered to 166 games. Total: 830
Fetched 500 games. Filtered to 166 games. Total: 996
Fetched 500 games. Filtered to 166 games. Total: 1162
Fetched 500 games. Filtered to 166 games. Total: 1328
Fetched 500 games. Filtered to 166 games. Total: 1494
Fetched 500 games. Filtered to 166 games. Total: 1660
Fetched 500 games. Filtered to 166 games. Total: 1826
Fetched 500 games. Filtered to 166 games. Total: 1992
Fetched 500 games. Filtered to 166 games. Total: 2158
Fetched 500 games. Filtered to 166 games. Total: 2324
Fetched 500 games. Filtered to 166 games. Total: 2490
Fetched 500 games. Filtered to 166 games. Total: 2656
Fetched 500 games. Filtered to 166 games. Total: 2822
Fetched 500 games. Filtered to 166 games. Total: 2988
Fetched 500 games. Filtered to 166

KeyboardInterrupt: 

In [ ]:
df.info()

## En TRAVAUX ##

In [21]:
from google.colab import files
df.to_parquet('df_gros.parquet')
files.download('df_gros.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_total2 = df_total.query('name.notna()', engine='python')

In [ ]:
df_total2["released"] = pd.to_datetime(df_total2["released"])

current_year = pd.Timestamp.now().year
df_total2['year'] = df_total2['released'].dt.year
df_total2 = df_total2[df_total2['year'] <= current_year]
df_total2['decennie'] = (df_total2['year'] // 10) * 10

<ipython-input-11-5ce05b42a014>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total2["released"] = pd.to_datetime(df_total2["released"])
<ipython-input-11-5ce05b42a014>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total2['year'] = df_total2['released'].dt.year
<ipython-input-11-5ce05b42a014>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [ ]:
df_total2.head(10)

,id,game_name,slug,name,metacritic,released,tba,updated,website,rating,rating_top,playtime,achievements_count,ratings_count,suggestions_count,game_series_count,reviews_count,platforms,developers,genres,publishers,esrb_rating,added_status_yet,added_status_owned,added_status_beaten,added_status_toplay,added_status_dropped,added_status_playing,year,decennie
0,3498,grand-theft-auto-v,grand-theft-auto-v,Grand Theft Auto V,97.0,2013-09-17,False,2020-09-23T07:10:53,http://www.rockstargames.com/V/,4.48,5.0,69.0,369.0,4289.0,426.0,9.0,4334.0,PC||Xbox Series S/X||PlayStation 5||PlayStatio...,Rockstar North,Action,Rockstar Games||1C-SoftClub,Mature,331.0,8298.0,3533.0,377.0,641.0,497.0,2013.0,2010.0
1,3328,the-witcher-3-wild-hunt,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,92.0,2015-05-18,False,2020-10-02T00:57:03,https://thewitcher.com/en/witcher3,4.67,5.0,50.0,204.0,3939.0,688.0,6.0,3996.0,PC||Xbox One||Nintendo Switch||PlayStation 4,CD PROJEKT RED,Action||Adventure||RPG,CD PROJEKT RED,Mature,635.0,6766.0,2759.0,517.0,485.0,642.0,2015.0,2010.0
2,4200,portal-2,portal-2,Portal 2,95.0,2011-04-18,False,2020-08-03T02:17:38,http://www.thinkwithportals.com/,4.61,5.0,11.0,152.0,3613.0,589.0,2.0,3645.0,Xbox One||PlayStation 3||PC||Xbox 360||Linux||...,Valve Software,Shooter||Puzzle,Electronic Arts||Valve,Everyone 10+,386.0,7506.0,3414.0,211.0,324.0,97.0,2011.0,2010.0
3,5286,tomb-raider,tomb-raider,Tomb Raider (2013),86.0,2013-03-05,False,2020-08-03T05:49:25,http://www.tombraider.com,4.06,4.0,11.0,148.0,2527.0,673.0,17.0,2542.0,PC||PlayStation 4||PlayStation 3||Xbox 360||Xb...,Crystal Dynamics,Action||Adventure,Square Enix,Mature,375.0,7299.0,2599.0,149.0,305.0,80.0,2013.0,2010.0
4,5679,the-elder-scrolls-v-skyrim,the-elder-scrolls-v-skyrim,The Elder Scrolls V: Skyrim,94.0,2011-11-11,False,2020-07-06T04:26:04,http://elderscrolls.com/,4.42,5.0,44.0,302.0,3033.0,625.0,8.0,3053.0,PC||PlayStation 3||Xbox 360||Nintendo Switch,Bethesda Softworks||Bethesda Game Studios,Action||RPG,Bethesda Softworks,Mature,299.0,6716.0,2313.0,246.0,801.0,271.0,2011.0,2010.0
5,4291,counter-strike-global-offensive,counter-strike-global-offensive,Counter-Strike: Global Offensive,81.0,2012-08-21,False,2020-08-07T06:18:15,http://blog.counter-strike.net/,3.57,4.0,59.0,179.0,2089.0,608.0,5.0,2104.0,PC||Xbox 360||PlayStation 3,Valve Software||Hidden Path Entertainment,Action||Shooter,Valve,Mature,163.0,7821.0,544.0,41.0,1056.0,388.0,2012.0,2010.0
6,12020,left-4-dead-2,left-4-dead-2,Left 4 Dead 2,89.0,2009-11-17,False,2020-12-02T13:36:02,http://www.l4d.com,4.10,4.0,9.0,166.0,2107.0,610.0,2.0,2116.0,PC||Xbox 360,Valve Software||Turtle Rock Studios,Action||Shooter,Electronic Arts||Valve,Mature,250.0,7708.0,1542.0,56.0,634.0,99.0,2009.0,2000.0
7,13536,portal,portal,Portal,90.0,2007-10-09,False,2020-03-08T18:54:32,http://www.whatistheorangebox.com/,4.52,5.0,4.0,15.0,3050.0,329.0,2.0,3069.0,macOS||Android||PC||Linux||PlayStation 3||Xbox...,Valve Software||NVIDIA Lightspeed Studios,Action||Adventure||Puzzle,Valve,Teen,247.0,6402.0,3088.0,126.0,215.0,45.0,2007.0,2000.0
8,4062,bioshock-infinite,bioshock-infinite,BioShock Infinite,94.0,2013-03-26,False,2020-08-03T02:19:32,http://www.bioshockinfinite.com/,4.39,5.0,12.0,370.0,2769.0,599.0,4.0,2790.0,PC||Nintendo Switch||Linux||PlayStation 3||Pla...,Aspyr Media||2K Australia||Irrational Games,Action||Shooter,2K Games||Aspyr,Mature,493.0,6242.0,2820.0,238.0,297.0,67.0,2013.0,2010.0
9,802,borderlands-2,borderlands-2,Borderlands 2,89.0,2012-09-18,False,2020-08-07T02:30:06,http://www.borderlands2.com/,4.05,4.0,10.0,302.0,2235.0,660.0,4.0,2246.0,PC||PlayStation 3||Xbox 360||PlayStation 4||ma...,Aspyr Media||2K||Gearbox Software||Sony Comput...,Action||Shooter||RPG,2K Games||Aspyr,Mature,388.0,7244.0,1495.0,114.0,788.0,188.0,2012.0,2010.0


In [ ]:
df_clean = df_total2.dropna(subset=['year', 'playtime','rating','genres'])
df_clean_genre = df_clean.dropna(subset=['genres'])

In [ ]:

# Séparer les genres en une liste en utilisant '||' comme séparateur
df_clean['genres_split'] = df_clean['genres'].str.split('\|\|')

# Afficher un échantillon des genres séparés pour vérifier le résultat
print("Genres séparés :")
print(df_clean['genres_split'].head())

# Créer une liste unique de genres
all_genres = set(genre for sublist in df_clean['genres_split'] for genre in sublist)

# Créer des colonnes pour chaque genre
for genre in all_genres:
    df_clean[genre] = df_clean['genres_split'].apply(lambda x: 1 if genre in x else 0)

# Supprimer la colonne temporaire 'genres_split'
df_clean = df_clean.drop(columns=['genres_split'])

<ipython-input-14-d42ea3630695>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['genres_split'] = df_clean['genres'].str.split('\|\|')


Genres séparés :
0                    [Action]
1    [Action, Adventure, RPG]
2           [Shooter, Puzzle]
3         [Action, Adventure]
4               [Action, RPG]
Name: genres_split, dtype: object


<ipython-input-14-d42ea3630695>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[genre] = df_clean['genres_split'].apply(lambda x: 1 if genre in x else 0)
<ipython-input-14-d42ea3630695>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[genre] = df_clean['genres_split'].apply(lambda x: 1 if genre in x else 0)
<ipython-input-14-d42ea3630695>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [ ]:

# Remplacer les valeurs nulles par une chaîne vide pour éviter les erreurs
df_clean['platforms'] = df_clean['platforms'].fillna('')

# Séparer les plateformes en une liste en utilisant '||' comme séparateur
df_clean['platforms_split'] = df_clean['platforms'].str.split('\|\|')

# Afficher un échantillon des plateformes séparées pour vérifier le résultat
print("Plateformes séparées :")
print(df_clean['platforms_split'].head())

# Créer une liste unique de plateformes
all_platforms = set(platform for sublist in df_clean['platforms_split'] for platform in sublist)

# Créer des colonnes pour chaque plateforme
for platform in all_platforms:
    df_clean[platform] = df_clean['platforms_split'].apply(lambda x: 1 if platform in x else 0)

# Supprimer la colonne temporaire 'platforms_split'
df_clean = df_clean.drop(columns=['platforms_split'])

Plateformes séparées :
0    [PC, Xbox Series S/X, PlayStation 5, PlayStati...
1       [PC, Xbox One, Nintendo Switch, PlayStation 4]
2    [Xbox One, PlayStation 3, PC, Xbox 360, Linux,...
3    [PC, PlayStation 4, PlayStation 3, Xbox 360, X...
4       [PC, PlayStation 3, Xbox 360, Nintendo Switch]
Name: platforms_split, dtype: object


In [ ]:
df_clean.head(5)

,id,game_name,slug,name,metacritic,released,tba,updated,website,rating,rating_top,playtime,achievements_count,ratings_count,suggestions_count,game_series_count,reviews_count,platforms,developers,genres,publishers,esrb_rating,added_status_yet,added_status_owned,added_status_beaten,added_status_toplay,added_status_dropped,added_status_playing,year,decennie,Educational,Casual,Fighting,Board Games,Strategy,Card,Indie,Platformer,RPG,Massively Multiplayer,Action,Sports,Simulation,Shooter,Family,Racing,Adventure,Arcade,Puzzle,Nintendo 3DS,,macOS,PS Vita,NES,Nintendo Switch,PlayStation,Xbox Series S/X,Game Boy,Atari 7800,SEGA Saturn,Genesis,Atari ST,Atari Flashback,Nintendo DS,Xbox One,SEGA Master System,Game Gear,Apple II,Commodore / Amiga,Jaguar,PlayStation 5,Xbox,Linux,Atari 2600,Dreamcast,Nintendo DSi,Game Boy Advance,PlayStation 4,PC,Web,PlayStation 2,Nintendo 64,PlayStation 3,iOS,GameCube,Classic Macintosh,Atari 8-bit,Neo Geo,SNES,Android,Xbox 360,SEGA CD,Wii,SEGA 32X,PSP,Atari 5200,Game Boy Color,Atari Lynx,Atari XEGS,Wii U,3DO
0,3498,grand-theft-auto-v,grand-theft-auto-v,Grand Theft Auto V,97.0,2013-09-17,False,2020-09-23T07:10:53,http://www.rockstargames.com/V/,4.48,5.0,69.0,369.0,4289.0,426.0,9.0,4334.0,PC||Xbox Series S/X||PlayStation 5||PlayStatio...,Rockstar North,Action,Rockstar Games||1C-SoftClub,Mature,331.0,8298.0,3533.0,377.0,641.0,497.0,2013.0,2010.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,3328,the-witcher-3-wild-hunt,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,92.0,2015-05-18,False,2020-10-02T00:57:03,https://thewitcher.com/en/witcher3,4.67,5.0,50.0,204.0,3939.0,688.0,6.0,3996.0,PC||Xbox One||Nintendo Switch||PlayStation 4,CD PROJEKT RED,Action||Adventure||RPG,CD PROJEKT RED,Mature,635.0,6766.0,2759.0,517.0,485.0,642.0,2015.0,2010.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4200,portal-2,portal-2,Portal 2,95.0,2011-04-18,False,2020-08-03T02:17:38,http://www.thinkwithportals.com/,4.61,5.0,11.0,152.0,3613.0,589.0,2.0,3645.0,Xbox One||PlayStation 3||PC||Xbox 360||Linux||...,Valve Software,Shooter||Puzzle,Electronic Arts||Valve,Everyone 10+,386.0,7506.0,3414.0,211.0,324.0,97.0,2011.0,2010.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,5286,tomb-raider,tomb-raider,Tomb Raider (2013),86.0,2013-03-05,False,2020-08-03T05:49:25,http://www.tombraider.com,4.06,4.0,11.0,148.0,2527.0,673.0,17.0,2542.0,PC||PlayStation 4||PlayStation 3||Xbox 360||Xb...,Crystal Dynamics,Action||Adventure,Square Enix,Mature,375.0,7299.0,2599.0,149.0,305.0,80.0,2013.0,2010.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5679,the-elder-scrolls-v-skyrim,the-elder-scrolls-v-skyrim,The Elder Scrolls V: Skyrim,94.0,2011-11-11,False,2020-07-06T04:26:04,http://elderscrolls.com/,4.42,5.0,44.0,302.0,3033.0,625.0,8.0,3053.0,PC||PlayStation 3||Xbox 360||Nintendo Switch,Bethesda Softworks||Bethesda Game Studios,Action||RPG,Bethesda Softworks,Mature,299.0,6716.0,2313.0,246.0,801.0,271.0,2011.0,2010.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_export = df_clean.drop(['slug', 'game_name', 'tba', 'updated', 'rating_top', 'achievements_count',
                           'publishers', 'added_status_yet', 'added_status_owned', 'added_status_beaten',
                           'added_status_toplay', 'added_status_dropped', 'added_status_playing'], axis=1)


In [ ]:
df_export.to_parquet('df_igdb.parquet', index=False)

In [ ]:
'''
# import pour ydata_profiling
import ydata_profiling
from ydata_profiling import ProfileReport
from ydata_profiling.utils.cache import cache_file

# creation d'un rapport exploratoire de notre dataframe

profile = ProfileReport(df_total2, title="Analyse Générale",html={'style': {'full_width': True}})

# Affichage de notre rapport exploratoire :

profile.to_notebook_iframe()
'''

'\n# import pour ydata_profiling\nimport ydata_profiling\nfrom ydata_profiling import ProfileReport\nfrom ydata_profiling.utils.cache import cache_file\n\n# creation d\'un rapport exploratoire de notre dataframe\n\nprofile = ProfileReport(df_total2, title="Analyse Générale",html={\'style\': {\'full_width\': True}})\n\n# Affichage de notre rapport exploratoire :\n\nprofile.to_notebook_iframe()\n'

In [ ]:
df_total2.shape

(449719, 30)

In [ ]:
#px.line(df_total2, x="released", y=df_total2.select_dtypes(include='number').columns, title="Titre")

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349229 entries, 0 to 664947
Columns: 101 entries, id to 3DO
dtypes: datetime64[ns](1), float64(17), int64(71), object(12)
memory usage: 271.8+ MB


In [ ]:
df_clean['genres'].head(10)

,genres
0,Action
1,Action||Adventure||RPG
2,Shooter||Puzzle
3,Action||Adventure
4,Action||RPG
5,Action||Shooter
6,Action||Shooter
7,Action||Adventure||Puzzle
8,Action||Shooter
9,Action||Shooter||RPG


In [ ]:
df_clean.shape

(349229, 101)

In [ ]:
df_clean.columns

Index(['id', 'game_name', 'slug', 'name', 'metacritic', 'released', 'tba',
       'updated', 'website', 'rating',
       ...
       'SEGA CD', 'Wii', 'SEGA 32X', 'PSP', 'Atari 5200', 'Game Boy Color',
       'Atari Lynx', 'Atari XEGS', 'Wii U', '3DO'],
      dtype='object', length=101)

In [ ]:
df_clean.head(10)

,id,game_name,slug,name,metacritic,released,tba,updated,website,rating,rating_top,playtime,achievements_count,ratings_count,suggestions_count,game_series_count,reviews_count,platforms,developers,genres,publishers,esrb_rating,added_status_yet,added_status_owned,added_status_beaten,added_status_toplay,added_status_dropped,added_status_playing,year,decennie,Educational,Casual,Fighting,Board Games,Strategy,Card,Indie,Platformer,RPG,Massively Multiplayer,Action,Sports,Simulation,Shooter,Family,Racing,Adventure,Arcade,Puzzle,Nintendo 3DS,,macOS,PS Vita,NES,Nintendo Switch,PlayStation,Xbox Series S/X,Game Boy,Atari 7800,SEGA Saturn,Genesis,Atari ST,Atari Flashback,Nintendo DS,Xbox One,SEGA Master System,Game Gear,Apple II,Commodore / Amiga,Jaguar,PlayStation 5,Xbox,Linux,Atari 2600,Dreamcast,Nintendo DSi,Game Boy Advance,PlayStation 4,PC,Web,PlayStation 2,Nintendo 64,PlayStation 3,iOS,GameCube,Classic Macintosh,Atari 8-bit,Neo Geo,SNES,Android,Xbox 360,SEGA CD,Wii,SEGA 32X,PSP,Atari 5200,Game Boy Color,Atari Lynx,Atari XEGS,Wii U,3DO
0,3498,grand-theft-auto-v,grand-theft-auto-v,Grand Theft Auto V,97.0,2013-09-17,False,2020-09-23T07:10:53,http://www.rockstargames.com/V/,4.48,5.0,69.0,369.0,4289.0,426.0,9.0,4334.0,PC||Xbox Series S/X||PlayStation 5||PlayStatio...,Rockstar North,Action,Rockstar Games||1C-SoftClub,Mature,331.0,8298.0,3533.0,377.0,641.0,497.0,2013.0,2010.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,3328,the-witcher-3-wild-hunt,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,92.0,2015-05-18,False,2020-10-02T00:57:03,https://thewitcher.com/en/witcher3,4.67,5.0,50.0,204.0,3939.0,688.0,6.0,3996.0,PC||Xbox One||Nintendo Switch||PlayStation 4,CD PROJEKT RED,Action||Adventure||RPG,CD PROJEKT RED,Mature,635.0,6766.0,2759.0,517.0,485.0,642.0,2015.0,2010.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4200,portal-2,portal-2,Portal 2,95.0,2011-04-18,False,2020-08-03T02:17:38,http://www.thinkwithportals.com/,4.61,5.0,11.0,152.0,3613.0,589.0,2.0,3645.0,Xbox One||PlayStation 3||PC||Xbox 360||Linux||...,Valve Software,Shooter||Puzzle,Electronic Arts||Valve,Everyone 10+,386.0,7506.0,3414.0,211.0,324.0,97.0,2011.0,2010.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,5286,tomb-raider,tomb-raider,Tomb Raider (2013),86.0,2013-03-05,False,2020-08-03T05:49:25,http://www.tombraider.com,4.06,4.0,11.0,148.0,2527.0,673.0,17.0,2542.0,PC||PlayStation 4||PlayStation 3||Xbox 360||Xb...,Crystal Dynamics,Action||Adventure,Square Enix,Mature,375.0,7299.0,2599.0,149.0,305.0,80.0,2013.0,2010.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5679,the-elder-scrolls-v-skyrim,the-elder-scrolls-v-skyrim,The Elder Scrolls V: Skyrim,94.0,2011-11-11,False,2020-07-06T04:26:04,http://elderscrolls.com/,4.42,5.0,44.0,302.0,3033.0,625.0,8.0,3053.0,PC||PlayStation 3||Xbox 360||Nintendo Switch,Bethesda Softworks||Bethesda Game Studios,Action||RPG,Bethesda Softworks,Mature,299.0,6716.0,2313.0,246.0,801.0,271.0,2011.0,2010.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,4291,counter-strike-global-offensive,counter-strike-global-offensive,Counter-Strike: Global Offensive,81.0,2012-08-21,False,2020-08-07T06:18:15,http://blog.counter-strike.net/,3.57,4.0,59.0,179.0,2089.0,608.0,5.0,2104.0,PC||Xbox 360||PlayStation 3,Valve Software||Hidden Path Entertainment,Action||Shooter,Valve,Mature,163.0,7821.0,544.0,41.0,1056.0,388.0,2012.0,2010.0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6,12020,left-4-dea

In [ ]:
df_clean.to_parquet('df_igdb.parquet')